# Data Cleaning, converting to prepared data set

(https://www.kaggle.com/c/nfl-big-data-bowl-2020)

The data that arrived is in one big csv formatted file, which contains 49 attributes and 682154 records, representing 31007 plays in 688 games. The opening lines of code are here to show the data, and look at some field info and data. For the most part, the columns are coded correctly, but some have to be changed. Also, it looks like we have some missing data that has to be addressed.

In [1]:
# All import insertions
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import isnull
from sklearn import preprocessing
import os    # For file paths
import re    # Regular Expressions

import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt
import time

In [2]:
# Opening the original file
df = pd.read_csv("../data/raw/train.csv", low_memory=False)

In [3]:
# A quick opening to look at the format of the data.
df.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [4]:
#Number of records in the dataframe
df.shape

(682154, 49)

In [5]:
# Number of unique plays in the data
df['PlayId'].nunique()

31007

In [6]:
# Number of unique games in the data
df['GameId'].nunique()

688

In [7]:
# Breaking down the week numbers so we know that breakdown. 
# It looks like we don't have a full season for 2019, but that shouldn't affect the data
print("2017: ")
print(df[df['Season'] == 2017]['Week'].unique())
print("2018: ")
print(df[df['Season'] == 2018]['Week'].unique())
print("2019: ")
print(df[df['Season'] == 2019]['Week'].unique())

2017: 
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
2018: 
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
2019: 
[ 1  2  3  4  5  6  7  8  9 10 11 12]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 49 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   GameId                  682154 non-null  int64  
 1   PlayId                  682154 non-null  int64  
 2   Team                    682154 non-null  object 
 3   X                       682154 non-null  float64
 4   Y                       682154 non-null  float64
 5   S                       682154 non-null  float64
 6   A                       682154 non-null  float64
 7   Dis                     682154 non-null  float64
 8   Orientation             682131 non-null  float64
 9   Dir                     682126 non-null  float64
 10  NflId                   682154 non-null  int64  
 11  DisplayName             682154 non-null  object 
 12  JerseyNumber            682154 non-null  int64  
 13  Season                  682154 non-null  int64  
 14  YardLine            

In [9]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
GameId,682154.0,2.017975e+09,7.844593e+05,2.017091e+09,2.017113e+09,2.018101e+09,2.019091e+09,2.019112e+09
PlayId,682154.0,2.017975e+13,7.844593e+09,2.017091e+13,2.017113e+13,2.018101e+13,2.019091e+13,2.019113e+13
X,682154.0,6.003686e+01,2.549931e+01,6.600000e-01,3.908000e+01,6.023000e+01,8.099000e+01,1.176500e+02
Y,682154.0,2.679146e+01,7.192919e+00,1.850000e+00,2.276000e+01,2.680000e+01,3.085000e+01,5.645000e+01
S,682154.0,2.634598e+00,1.404773e+00,0.000000e+00,1.520000e+00,2.510000e+00,3.620000e+00,9.410000e+00
A,682154.0,1.701739e+00,9.913802e-01,0.000000e+00,9.500000e-01,1.550000e+00,2.310000e+00,1.471000e+01
Dis,682154.0,2.781932e-01,1.452182e-01,0.000000e+00,1.600000e-01,2.700000e-01,3.900000e-01,1.390000e+00
Orientation,682131.0,1.799967e+02,1.027946e+02,0.000000e+00,9.006000e+01,1.796400e+02,2.697800e+02,3.600000e+02
Dir,682126.0,1.800406e+02,1.061895e+02,0.000000e+00,9.015000e+01,1.801800e+02,2.702600e+02,3.600000e+02
NflId,682154.0,2.369743e+06,6.053648e+05,2.220000e+02,2.532989e+06,2.550541e+06,2.556445e+06,2.563364e+06


## Creating the new predictor variable
The data came with a predictor variable in the number of yards collected. I wanted to create a new predictor variable for this study, so I created the binary column of whether or not the play made a first down or not. For this, if the number of yards gained is greater than, or equal to the number yards needed to get the first down, the result is a first down success (1), otherwise, it's marked as a fail (0)

In [10]:
# The original predictor variable
df['Yards'].describe()

count    682154.000000
mean          4.227626
std           6.449866
min         -15.000000
25%           1.000000
50%           3.000000
75%           6.000000
max          99.000000
Name: Yards, dtype: float64

In [11]:
# Add the column if the first down was met, new predictor variable
df['isFirstDown'] = np.where(df['Distance']<=df['Yards'], 1, 0)

In [12]:
# Grab 20 random records to test the new predictive variable
df[['Down', 'Distance', 'Yards', 'isFirstDown']].sample(n=20, random_state=52)

,Down,Distance,Yards,isFirstDown
348444,2,26,6,0
257609,2,6,3,0
598402,2,10,4,0
169077,1,10,0,0
297095,1,14,0,0
365572,1,10,14,1
397421,2,5,23,1
477100,1,10,3,0
365563,1,10,14,1
352907,3,1,1,1


## Fixing the Wind Speed and Wind Direction fields
The only real fields that needs some extended fixing was the Wind Speed and the Wind Direction. The data collected was transposed in certain scenarios, so it needs to be adjusted before we do anything else. Some columns have the wind direction in compass notation in the wind speed column, so to fix this, we got an index list of all the rows that have compass directions and swapped fields. This looked to fix the problem.

In [13]:
df['WindSpeed'].unique()

array(['8', '6', '10', '9', '11', nan, '7', '5', '2', '12', '1', '3', '4',
       '13', '0', 'SSW', '14', '15', '17', '18', '16', '11-17', '23',
       '14-23', '13 MPH', '24', '12-22', '4 MPh', '15 gusts up to 25',
       '10MPH', '10mph', '22', 'E', '7 MPH', 'Calm', '6 mph', '19', 'SE',
       '20', '10-20', '12mph', '6mph', '9mph', 'SSE', '14 Gusting to 24',
       '6 mph, Gusts to 10', '2 mph, gusts to 5', '12 mph',
       '9 mph, gusts to 13', '10 mph, gusts to 15'], dtype=object)

In [14]:
# Fixing the fields that are in the wrong columns
from itertools import compress
dfindex = df['WindSpeed'].str.contains("[NESW]+", regex=True, na=False)
dfres = list(compress(range(len(dfindex)), dfindex)) 

In [15]:
# Swap the incorrect fields
for (k) in dfres:
    ws = df.iloc[k]['WindDirection']
    wd = df.iloc[k]['WindSpeed']
    df.at[k, 'WindDirection'] = wd
    df.at[k, 'WindSpeed'] = ws

In [16]:
df['WindSpeed'].unique()

array(['8', '6', '10', '9', '11', nan, '7', '5', '2', '12', '1', '3', '4',
       '13', '0', '14', '15', '17', '18', '16', '11-17', '23', '14-23',
       '13 MPH', '24', '12-22', '4 MPh', '15 gusts up to 25', '10MPH',
       '10mph', '22', '7 MPH', 'Calm', '6 mph', '19', '20', '10-20',
       '12mph', '6mph', '9mph', '14 Gusting to 24', '6 mph, Gusts to 10',
       '2 mph, gusts to 5', '12 mph', '9 mph, gusts to 13',
       '10 mph, gusts to 15'], dtype=object)

In [17]:
# Filling the missing fields
df['WindSpeed']=df['WindSpeed'].fillna(0)
df['WindDirection']=df['WindDirection'].fillna('No Wind')

In [18]:
# Correcting the fields that have spare texts
df['WindSpeed'].replace(to_replace="^[\D]*(\d+).*$", value=r"\1", regex=True, inplace=True)
df['WindSpeed'].replace(to_replace="Calm", value=r"0", regex=True, inplace=True)

In [19]:
df['WindSpeed'].unique()

array(['8', '6', '10', '9', '11', 0, '7', '5', '2', '12', '1', '3', '4',
       '13', '0', '14', '15', '17', '18', '16', '23', '24', '22', '19',
       '20'], dtype=object)

## Fixing the timestamps as date types
The Majority of the fields were correctly labeled as integeres, floats and strings, but there are several date fields that need to be changed before we continue.

In [20]:
df['TimeHandoff'] = df['TimeHandoff'].astype('datetime64')
df['TimeSnap'] = df['TimeSnap'].astype('datetime64')
df['PlayerBirthDate'] = df['PlayerBirthDate'].astype('datetime64')
df['GameClock'] = df['GameClock'].astype('datetime64')

df.info() # .dtypes 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 50 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   GameId                  682154 non-null  int64         
 1   PlayId                  682154 non-null  int64         
 2   Team                    682154 non-null  object        
 3   X                       682154 non-null  float64       
 4   Y                       682154 non-null  float64       
 5   S                       682154 non-null  float64       
 6   A                       682154 non-null  float64       
 7   Dis                     682154 non-null  float64       
 8   Orientation             682131 non-null  float64       
 9   Dir                     682126 non-null  float64       
 10  NflId                   682154 non-null  int64         
 11  DisplayName             682154 non-null  object        
 12  JerseyNumber            682154

In [21]:
#df['GameClock'].head()

## Creating a numerical field for home/away
The home and away column is text, so I made it into a binary column

In [22]:
df.insert(df.columns.get_loc('Team') + 1, 'isHomeTeam', np.where(df['Team'] == 'home', 1, 0 ))
#df[['Team','isHomeTeam']].head(22)

## Fixing the abbreviated team names
For the team abbriviations, some teams had more than one listed in some columns, so this was normalized manually to make sure the team name abbreviations are consistent across the board.

In [23]:
df['HomeTeamAbbr'].sort_values().unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [24]:
# Fixing the abbreviations, some of them differed from the other
df['FieldPosition']=df['FieldPosition'].fillna('Midfield')
df['FieldPosition']=df['FieldPosition'].replace('ARZ','ARI')
df['FieldPosition']=df['FieldPosition'].replace('BLT','BAL')
df['FieldPosition']=df['FieldPosition'].replace('CLV','CLE')
df['FieldPosition']=df['FieldPosition'].replace('HST','HOU')

df['FieldPosition'].sort_values().unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'Midfield', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT',
       'SEA', 'SF', 'TB', 'TEN', 'WAS'], dtype=object)

In [25]:
df['HomeTeamAbbr'].sort_values().unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [26]:
df['VisitorTeamAbbr'].sort_values().unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [27]:
# Fixing the abbreviations, some of them differed from the other
df['PossessionTeam']=df['PossessionTeam'].replace('ARZ','ARI')
df['PossessionTeam']=df['PossessionTeam'].replace('BLT','BAL')
df['PossessionTeam']=df['PossessionTeam'].replace('CLV','CLE')
df['PossessionTeam']=df['PossessionTeam'].replace('HST','HOU')

df['PossessionTeam'].sort_values().unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)

In [28]:
teamabtxt=df['HomeTeamAbbr'].sort_values().unique()
dctTeamab={}
cnt=1
for x in teamabtxt:
    dctTeamab[x]=cnt
    cnt=cnt+1

df['FieldPosition']=df['FieldPosition'].fillna('Midfield')
dctTeamab['Midfield']=0 #Mid-field

# Populate the numerical fields
df.insert(df.columns.get_loc('FieldPosition') + 1, 'FieldPositionCode', df['FieldPosition'].map(lambda x: dctTeamab[x]) )
df.insert(df.columns.get_loc('HomeTeamAbbr') + 1, 'HomeTeamAbbrCode', df['HomeTeamAbbr'].map(lambda x: dctTeamab[x]) )
df.insert(df.columns.get_loc('VisitorTeamAbbr') + 1, 'VisitorTeamAbbrCode', df['VisitorTeamAbbr'].map(lambda x: dctTeamab[x]) )
df.insert(df.columns.get_loc('PossessionTeam') + 1, 'PossessionTeamAbbrCode', df['PossessionTeam'].map(lambda x: dctTeamab[x]) )

## Calculating the number of yards to the end zone, based on the yardline and which team has the bowl
A new attribute is being created called the Yards to Touchdown. It helps clarify how many yards are needed for the touchdown, instead of using the position field, which has the location and you need which team has the ball. So for example, if the team is 20 yards from the end zone, the Field position should read 20, which is correct. But if the possessing team has 60 yards to go, the field position would read 40, because at yard markers displayed go descending from midfield (50 yards between each end zone). 

In [29]:
df.insert(df.columns.get_loc('Yards') + 1, 'YardsToTouchdown', np.NaN )

In [30]:
df['YardsToTouchdown'] = np.where( df['PossessionTeam'] == df['FieldPosition'], (50 + (50 - df['YardLine'])), df['YardLine'])

In [31]:
df['YardLine'].unique()

array([35, 43,  2, 25, 29, 19, 10, 50,  9, 30, 12,  8, 45, 18, 22, 48, 36,
       20, 11,  7, 23, 37, 40,  4, 28, 46, 14,  1,  5, 13, 15, 49, 21, 42,
       33, 32, 34, 31, 41, 38, 17, 47, 26,  6, 44, 39, 24, 16,  3, 27],
      dtype=int64)

In [32]:
df['YardsToTouchdown'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int64)

## Calculating the number of seconds from snap to handoff
We have two attributes of the exact time the ball was snapped, and the time the ball was handed off to the running back. A new field was added to the data as the difference between the two. If the number is 0, the ball was snapped directly to the running back, or the quarterback is running with the ball instead of throwing the ball.

In [33]:
df[['TimeHandoff','TimeSnap']].head()

,TimeHandoff,TimeSnap
0,2017-09-08 00:44:06,2017-09-08 00:44:05
1,2017-09-08 00:44:06,2017-09-08 00:44:05
2,2017-09-08 00:44:06,2017-09-08 00:44:05
3,2017-09-08 00:44:06,2017-09-08 00:44:05
4,2017-09-08 00:44:06,2017-09-08 00:44:05


In [34]:
df.insert(df.columns.get_loc('TimeSnap') + 1, 'TimeSnapDiff', np.NaN )
df['TimeSnapDiff'] = (pd.to_datetime(df['TimeHandoff'])-pd.to_datetime(df['TimeSnap']))
df['TimeSnapDiff']=(df['TimeSnapDiff']).dt.total_seconds()

In [35]:
df[['TimeHandoff','TimeSnap','TimeSnapDiff']].sample(n=10, random_state=68)

,TimeHandoff,TimeSnap,TimeSnapDiff
164175,2017-11-23 18:37:57,2017-11-23 18:37:55,2.0
377225,2018-10-30 00:53:11,2018-10-30 00:53:10,1.0
119164,2017-10-29 20:55:41,2017-10-29 20:55:40,1.0
3532,2017-09-10 18:23:04,2017-09-10 18:23:03,1.0
458372,2018-12-09 23:39:36,2018-12-09 23:39:35,1.0
135181,2017-11-10 01:32:23,2017-11-10 01:32:22,1.0
639010,2019-11-04 00:04:34,2019-11-04 00:04:31,3.0
428296,2018-11-25 23:29:06,2018-11-25 23:29:04,2.0
559365,2019-09-29 20:09:38,2019-09-29 20:09:37,1.0
552938,2019-09-22 21:27:05,2019-09-22 21:27:03,2.0


In [36]:
df['TimeSnapDiff'].value_counts(dropna = False).sort_index()

0.0      5610
1.0    489456
2.0    185834
3.0       968
4.0       176
5.0        66
7.0        44
Name: TimeSnapDiff, dtype: int64

## Calculating Age
Instead of using the birth date, we are creating an age attribute by taking the time of handoff, and subtracting it from the player's birth date.

In [37]:
df['PlayerBirthDate'] = pd.to_datetime(df['PlayerBirthDate'], utc=True)
df['TimeHandoff'] = pd.to_datetime(df['TimeHandoff'], utc=True)
df[['TimeHandoff','PlayerBirthDate']].head()

,TimeHandoff,PlayerBirthDate
0,2017-09-08 00:44:06+00:00,1988-12-29 00:00:00+00:00
1,2017-09-08 00:44:06+00:00,1989-03-25 00:00:00+00:00
2,2017-09-08 00:44:06+00:00,1989-01-21 00:00:00+00:00
3,2017-09-08 00:44:06+00:00,1982-11-22 00:00:00+00:00
4,2017-09-08 00:44:06+00:00,1987-08-17 00:00:00+00:00


In [38]:
df.insert(df.columns.get_loc('PlayerBirthDate') + 1, 'Age', np.NaN )

In [39]:
df['Age'] =  (((pd.to_datetime(df['TimeHandoff']) - pd.to_datetime(df['PlayerBirthDate'])).dt.days)/365)
df['Age'] = df['Age'].astype('int')

In [40]:
df[['TimeHandoff','PlayerBirthDate','Age']].head()

,TimeHandoff,PlayerBirthDate,Age
0,2017-09-08 00:44:06+00:00,1988-12-29 00:00:00+00:00,28
1,2017-09-08 00:44:06+00:00,1989-03-25 00:00:00+00:00,28
2,2017-09-08 00:44:06+00:00,1989-01-21 00:00:00+00:00,28
3,2017-09-08 00:44:06+00:00,1982-11-22 00:00:00+00:00,34
4,2017-09-08 00:44:06+00:00,1987-08-17 00:00:00+00:00,30


## Converting score fields to lead/deficit field
Two of the fields provided are the score of the game the moment the play is being made. A new field was created to create a binary of whether the team possessing has the lead, or is in a deficit, and by how much. 

In [41]:
df[['HomeScoreBeforePlay','VisitorScoreBeforePlay','isHomeTeam']].sample(n=5, random_state=52)

,HomeScoreBeforePlay,VisitorScoreBeforePlay,isHomeTeam
348444,0,21,0
257609,15,10,1
598402,0,13,0
169077,23,16,0
297095,3,3,0


In [42]:
df.insert(df.columns.get_loc('VisitorScoreBeforePlay') + 1, 'LeadDeficit', np.where(df['isHomeTeam'] == 1, df['HomeScoreBeforePlay'] - df['VisitorScoreBeforePlay'], df['VisitorScoreBeforePlay'] - df['HomeScoreBeforePlay'] ) )

In [43]:
df.insert(df.columns.get_loc('LeadDeficit') + 1, 'IsPossTeamInLead', np.where(df['LeadDeficit'] < 0, 1, 0 ) )

In [44]:
df.insert(df.columns.get_loc('IsPossTeamInLead') + 1, 'IsPossTeamInDeficit', np.where(df['LeadDeficit'] > 0, 1, 0 ) )

In [45]:
df[['HomeScoreBeforePlay','VisitorScoreBeforePlay','isHomeTeam','LeadDeficit']].sample(n=5, random_state=52)

,HomeScoreBeforePlay,VisitorScoreBeforePlay,isHomeTeam,LeadDeficit
348444,0,21,0,21
257609,15,10,1,5
598402,0,13,0,13
169077,23,16,0,-7
297095,3,3,0,0


In [46]:
df['LeadDeficit'].value_counts(dropna = False).sort_index()

-49     88
-44    220
-43     88
-42    143
-41     99
      ... 
 41     99
 42    143
 43     88
 44    220
 49     88
Name: LeadDeficit, Length: 91, dtype: int64

## Converting the player height to inches
Height was given as a text column, so several columns were created to simplify the calculation from text to the number of inches the player stands.

In [47]:
df['PlayerHeight'].unique()
df.insert(df.columns.get_loc('PlayerHeight') + 1, 'PlayerHeightInches', np.NaN )
df.insert(df.columns.get_loc('PlayerHeight') + 1, 'PlayerHeightIn', np.NaN )
df.insert(df.columns.get_loc('PlayerHeight') + 1, 'PlayerHeightFt', np.NaN )

In [48]:
df['PlayerHeightFt'] = df['PlayerHeight'].str.extract('^(\d+)', expand=True)
df['PlayerHeightIn'] = df['PlayerHeight'].str.extract('(\d+)$', expand=True)

In [49]:
df['PlayerHeightFt']=df['PlayerHeightFt'].astype('int64')
df['PlayerHeightIn']=df['PlayerHeightIn'].astype('int64')

In [50]:
df['PlayerHeightInches'] = (df['PlayerHeightFt'].mul(12) + df['PlayerHeightIn']).astype('int64')

In [51]:
df[['PlayerHeight', 'PlayerHeightFt', 'PlayerHeightIn', 'PlayerHeightInches']].sample(n=10, random_state=34)

,PlayerHeight,PlayerHeightFt,PlayerHeightIn,PlayerHeightInches
478618,6-6,6,6,78
117392,5-11,5,11,71
575601,6-2,6,2,74
425934,6-3,6,3,75
557854,6-4,6,4,76
632881,6-5,6,5,77
213322,6-5,6,5,77
570202,6-4,6,4,76
116284,6-4,6,4,76
446294,6-1,6,1,73


In [52]:
df['PlayerHeight'].value_counts(dropna = False).sort_index()

5-10     32286
5-11     54009
5-6        522
5-7        945
5-8       4638
5-9      12251
6-0      67819
6-1      82336
6-2      71347
6-3     108240
6-4      90139
6-5      83706
6-6      47488
6-7      16099
6-8       7646
6-9       2683
Name: PlayerHeight, dtype: int64

In [53]:
df['PlayerHeightInches'].value_counts(dropna = False).sort_index()

66       522
67       945
68      4638
69     12251
70     32286
71     54009
72     67819
73     82336
74     71347
75    108240
76     90139
77     83706
78     47488
79     16099
80      7646
81      2683
Name: PlayerHeightInches, dtype: int64

## Filling the empty fields
Many blank fields needs to be addressed, most of them are text fields which may already be tossed later, but still needs to be cleaned before anything else. 

In [54]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Orientation             23
Dir                     28
OffenseFormation        88
DefendersInTheBox       22
StadiumType          41690
GameWeather          60170
Temperature          63646
Humidity              6160
dtype: int64

In [55]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

Orientation has 23 null values
Dir has 28 null values
OffenseFormation has 88 null values
DefendersInTheBox has 22 null values
StadiumType has 41690 null values
GameWeather has 60170 null values
Temperature has 63646 null values
Humidity has 6160 null values


In [56]:
# Orientation Blank Fills (mean)
print("Before: ", df['Orientation'].unique())
df['Orientation']=df['Orientation'].fillna(df['Orientation'].mean())
print("After:  ", df['Orientation'].unique())

Before:  [ 81.99  27.61   3.01 ... 231.96 263.02 243.55]
After:   [ 81.99  27.61   3.01 ... 231.96 263.02 243.55]


In [57]:
# Dir Blank Fills  (Mean)
print("Before: ", df['Dir'].unique())
df['Dir']=df['Dir'].fillna(df['Dir'].mean())
print("After:  ", df['Dir'].unique())

Before:  [177.18 198.7  202.73 ... 117.84 253.69  80.28]
After:   [177.18 198.7  202.73 ... 117.84 253.69  80.28]


In [58]:
# Offensive Formations Blank Fills (Empty Text)
print("Before: ", df['OffenseFormation'].unique())
df['OffenseFormation']=df['OffenseFormation'].fillna('UNKNOWN')
print("After:  ",df['OffenseFormation'].unique())

Before:  ['SHOTGUN' 'SINGLEBACK' 'JUMBO' 'PISTOL' 'I_FORM' 'ACE' 'WILDCAT' nan
 'EMPTY']
After:   ['SHOTGUN' 'SINGLEBACK' 'JUMBO' 'PISTOL' 'I_FORM' 'ACE' 'WILDCAT'
 'UNKNOWN' 'EMPTY']


In [59]:
# Number of Defenders in the Box Blank Fills (Mean)
print("Before: ", df['DefendersInTheBox'].sort_values().unique())
df['DefendersInTheBox']=df['DefendersInTheBox'].fillna(round(df['DefendersInTheBox'].mean()))
df['DefendersInTheBox'] = df['DefendersInTheBox'].astype('int64')
print("After:  ", df['DefendersInTheBox'].sort_values().unique())

Before:  [ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]
After:   [ 2  3  4  5  6  7  8  9 10 11]


In [60]:
df[df['Humidity'].isnull()]['Stadium'].unique()
df['Humidity'].fillna(0, inplace=True)
#Humidity has too many 0's and missing fields, remove (Bottom of script)
#df.drop(['Humidity'], axis=1, inplace=True)

In [61]:
nullstadium = list(df[df['Temperature'].isnull()]['Stadium'].unique())

In [62]:
print ("Data before filling nulls with the mean per stadium")
for stdm in nullstadium:
    print('-' * 50)
    print(stdm)
    print(df[(df['Stadium'] == stdm)]['Temperature'].value_counts(dropna = False).sort_index())
    print("Mean: ",df[(df['Stadium'] == stdm)]['Temperature'].mean())
    print('-' * 50)
    print("\n")

Data before filling nulls with the mean per stadium
--------------------------------------------------
AT&T Stadium
54.0     1144
58.0     1166
67.0     1254
70.0      968
74.0      968
NaN     15862
Name: Temperature, dtype: int64
Mean:  64.148
--------------------------------------------------


--------------------------------------------------
U.S. Bank Stadium
70.0     1078
NaN     20526
Name: Temperature, dtype: int64
Mean:  70.0
--------------------------------------------------


--------------------------------------------------
University of Phoenix Stadium
71.0     968
79.0    1848
NaN     4796
Name: Temperature, dtype: int64
Mean:  76.25
--------------------------------------------------


--------------------------------------------------
Mercedes-Benz Stadium
52.0      968
59.0      924
60.0      770
65.0     1012
66.0      990
67.0      990
NaN     13420
Name: Temperature, dtype: int64
Mean:  61.63813229571984
--------------------------------------------------


--------

In [63]:
# For each stadium that is missing temperature values, replace missing values with the average
for stdm in nullstadium:
    m1 = (df['Stadium'] == stdm)
    m1m = round(df.loc[m1,'Temperature'].mean(),0)
    df.loc[m1,'Temperature'] = df.loc[m1,'Temperature'].fillna(m1m)

In [64]:
print ("Data after filling nulls with the mean per stadium")
for stdm in nullstadium:
    print('-' * 50)
    print(stdm)
    print(df[(df['Stadium'] == stdm)]['Temperature'].value_counts(dropna = False).sort_index())
    print("Mean: ",df[(df['Stadium'] == stdm)]['Temperature'].mean())
    print('-' * 50)
    print("\n")

Data after filling nulls with the mean per stadium
--------------------------------------------------
AT&T Stadium
54.0     1144
58.0     1166
64.0    15862
67.0     1254
70.0      968
74.0      968
Name: Temperature, dtype: int64
Mean:  64.03810504634397
--------------------------------------------------


--------------------------------------------------
U.S. Bank Stadium
70.0    21604
Name: Temperature, dtype: int64
Mean:  70.0
--------------------------------------------------


--------------------------------------------------
University of Phoenix Stadium
71.0     968
76.0    4796
79.0    1848
Name: Temperature, dtype: int64
Mean:  76.09248554913295
--------------------------------------------------


--------------------------------------------------
Mercedes-Benz Stadium
52.0      968
59.0      924
60.0      770
62.0    13420
65.0     1012
66.0      990
67.0      990
Name: Temperature, dtype: int64
Mean:  61.89273356401384
--------------------------------------------------




In [65]:
df[(df['Stadium'] == 'AT&T Stadium')]['Temperature'].isna().sum()

0

In [66]:
# Looking at the GameWeather field for nulls and for cleaning
df[df['GameWeather'].isnull()]['Stadium'].unique()

array(['AT&T Stadium', 'Wembley Stadium', 'Mercedes-Benz Stadium',
       'University of Phoenix Stadium', 'EverBank Field',
       'Hard Rock Stadium', 'Everbank Field', 'MetLife',
       'FirstEnergy Stadium', 'TIAA Bank Field', 'Ford Field',
       'StubHub Center', 'Arrowhead Stadium', 'State Farm Stadium',
       'Dignity Health Sports Park', 'Lambeau Field'], dtype=object)

In [67]:
df['GameWeather'].unique()

array(['Clear and warm', 'Sun & clouds', 'Sunny', 'Controlled Climate',
       'Mostly Sunny', 'Clear', nan, 'Indoor', 'Mostly Cloudy',
       'Mostly Coudy', 'Partly sunny', 'Partly Cloudy', 'Cloudy',
       'Sunny, highs to upper 80s', 'Indoors', 'Light Rain', 'Showers',
       'Partly cloudy', 'Partly Sunny', '30% Chance of Rain',
       'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
       'Rain', 'Cloudy, fog started developing in 2nd quarter', 'Coudy',
       'Rain likely, temps in low 40s.', 'Cold', 'N/A (Indoors)',
       'Clear skies', 'cloudy', 'Fair', 'Mostly cloudy',
       'Cloudy, chance of rain', 'Heavy lake effect snow', 'Party Cloudy',
       'Cloudy, light snow accumulating 1-3"', 'Cloudy and cold', 'Snow',
       'Hazy', 'Scattered Showers', 'Cloudy and Cool', 'N/A Indoor',
       'Rain Chance 40%', 'Clear and sunny', 'Mostly sunny',
       'Sunny and warm', 'Partly clear', 'Cloudy, 50% change of rain',
       'Clear and Sunny', '

In [68]:
df[df['GameWeather'].isnull()]['StadiumType'].unique()
#Gameweather is too inconsistent,  can remove
df['GameWeather'].fillna("Unknown", inplace=True)
#df.drop(['GameWeather'], axis=1, inplace=True)

## Cleaning the Stadium Column for consistency
The field, Stadium Type was way too large and inconsistent. It needs to be cleaned up, and broken down into a different set of categories

In [69]:
df['StadiumType']=df['StadiumType'].replace('Oudoor','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Outdoors','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Outddors','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Outdor','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Ourdoor','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Outside','Outdoor')
df['StadiumType']=df['StadiumType'].replace('OUTDOOR','Outdoor')

df['StadiumType']=df['StadiumType'].replace('Indoor','Indoors')
df['StadiumType']=df['StadiumType'].replace('indoor','Indoors')

df['StadiumType']=df['StadiumType'].replace('Heinz Field','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Cloudy','Outdoor')
df['StadiumType']=df['StadiumType'].replace('Indoor, roof open','Indoor, Open Roof')

df['StadiumType']=df['StadiumType'].replace('Bowl','Outdoor')

df['StadiumType']=df['StadiumType'].replace('Indoor, Roof Closed','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Closed Dome','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Dome, closed','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Retr. Roof-Closed','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Retr. Roof - Closed','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Retr. Roof Closed','Domed, closed')

df['StadiumType']=df['StadiumType'].replace('Dome','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Domed','Domed, closed')
df['StadiumType']=df['StadiumType'].replace('Retractable Roof - Closed','Domed, closed')


df['StadiumType']=df['StadiumType'].replace('Domed, open','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Indoor, Open Roof','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Indoor, Open Roof','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Retr. Roof-Open','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Retr. Roof - Open','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Open','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Outdoor Retr Roof-Open','Domed, Open')
df['StadiumType']=df['StadiumType'].replace('Retractable Roof','Domed, Open')

df['StadiumType'].value_counts(dropna = False)

Outdoor          463804
Indoors           77132
Domed, closed     57640
Domed, Open       41888
NaN               41690
Name: StadiumType, dtype: int64

In [70]:
df[df['Stadium'] == 'University of Phoenix Stadium']['StadiumType'].unique()

array(['Domed, closed', 'Domed, Open'], dtype=object)

In [71]:
# These Stadiums are blank, so we will rely on what was placed prior. If not, Wikipedia will confirm if the stadium is a dome or not
df[df['StadiumType'].isnull()]['Stadium'].value_counts(dropna = False)

MetLife Stadium               21406
StubHub Center                14630
Dignity Health Sports Park     4686
TIAA Bank Field                 968
Name: Stadium, dtype: int64

In [72]:
## MetLife Stadium - Home of the New York Giants and Jets
df[df['Stadium'] == 'MetLife Stadium']['StadiumType'].unique()  ## array([nan, 'Outdoor'], dtype=object)
df[df['Stadium'] == 'MetLife Stadium'] = df[df['Stadium'] == 'MetLife Stadium'].replace(np.nan,'Outdoor')
df[df['Stadium'] == 'MetLife Stadium']['StadiumType'].unique()

array(['Outdoor'], dtype=object)

In [73]:
## StubHub Center, now Dignity Health Sports Park, home of the Chargers
df[df['Stadium'] == 'StubHub Center']['StadiumType'].unique()  # array([nan], dtype=object)
df[df['Stadium'] == 'Dignity Health Sports Park']['StadiumType'].unique()
df[df['Stadium'] == 'StubHub Center'] = df[df['Stadium'] == 'StubHub Center'].replace(np.nan,'Outdoor')
df[df['Stadium'] == 'Dignity Health Sports Park'] = df[df['Stadium'] == 'Dignity Health Sports Park'].replace(np.nan,'Outdoor')
df[df['Stadium'] == 'StubHub Center']['StadiumType'].unique()  # array([nan], dtype=object)
df[df['Stadium'] == 'Dignity Health Sports Park']['StadiumType'].unique()

array(['Outdoor'], dtype=object)

In [74]:
# TIAA Bank Field, Home of the Jacksonville Jaguars
df[df['Stadium'] == 'TIAA Bank Field']['StadiumType'].unique()  ## array(['Open', 'Outdoor', nan, 'Bowl'], dtype=object)
df[df['Stadium'] == 'TIAA Bank Field'] = df[df['Stadium'] == 'TIAA Bank Field'].replace(np.nan,'Outdoor')
df[df['Stadium'] == 'TIAA Bank Field']['StadiumType'].unique()

array(['Domed, Open', 'Outdoor'], dtype=object)

In [75]:
df[df['StadiumType'].isnull()]['Stadium'].value_counts()

Series([], Name: Stadium, dtype: int64)

In [76]:
StadiumTypesCodes = {'Outdoor':1, 'Indoors':2, 'Domed, closed':3, 'Domed, Open':4}
df.insert(df.columns.get_loc('StadiumType') + 1, 'StadiumTypeCode', df['StadiumType'].map(StadiumTypesCodes ))

In [77]:
df[['StadiumType','StadiumTypeCode']].sample(n=10, random_state=99)

,StadiumType,StadiumTypeCode
429747,Indoors,2
389379,Indoors,2
370007,Outdoor,1
36447,Indoors,2
320504,Outdoor,1
423622,"Domed, closed",3
473503,Indoors,2
222549,Outdoor,1
175390,Outdoor,1
125752,"Domed, Open",4


In [78]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum().sum()

0.0

In [79]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

# Extracting the numbers from the offensive and defensive line
Looking at the OffensePersonnel and DefensePersonnel fields, there was a consideration to extract the numbers individually. There's another way of doing it, which is normalizing the other players and count that, so these attributes can be removed.

In [80]:
df['DefensePersonnel'].value_counts(dropna = False)

4 DL, 2 LB, 5 DB          177188
4 DL, 3 LB, 4 DB          173250
3 DL, 4 LB, 4 DB          110418
2 DL, 4 LB, 5 DB           81378
3 DL, 3 LB, 5 DB           74932
2 DL, 3 LB, 6 DB           17336
4 DL, 1 LB, 6 DB           10450
5 DL, 2 LB, 4 DB            7084
4 DL, 4 LB, 3 DB            6490
3 DL, 2 LB, 6 DB            5258
5 DL, 3 LB, 3 DB            3212
1 DL, 4 LB, 6 DB            2200
5 DL, 4 LB, 2 DB            1672
3 DL, 5 LB, 3 DB            1672
6 DL, 3 LB, 2 DB            1408
6 DL, 4 LB, 1 DB            1364
5 DL, 1 LB, 5 DB            1100
1 DL, 5 LB, 5 DB             924
6 DL, 2 LB, 3 DB             880
2 DL, 5 LB, 4 DB             616
1 DL, 3 LB, 7 DB             506
2 DL, 2 LB, 7 DB             418
4 DL, 5 LB, 2 DB             352
5 DL, 5 LB, 1 DB             330
3 DL, 1 LB, 7 DB             286
2 DL, 4 LB, 4 DB, 1 RB       220
1 DL, 4 LB, 5 DB, 1 RB       154
5 DL, 3 LB, 2 DB, 1 OL       154
0 DL, 4 LB, 7 DB             132
0 DL, 5 LB, 6 DB             132
1 DL, 3 LB

In [81]:
# Both text fields are redundant after the normalization data, so it can be removed
#df.drop(['OffensePersonnel', 'DefensePersonnel'], axis=1, inplace=True)

## Fixing other fields that are string to integer
Some of the string fields need to be factorized to categorical integers. Most have been done in the process, so the only one outstanding is the play direction. 

In [82]:
#df['PlayDirection'].unique()
df.insert(df.columns.get_loc('PlayDirection') + 1, 'isPlayDirectionLeft', np.where(df['PlayDirection'] == 'left', 1, 0 ) )

In [83]:
df.insert(df.columns.get_loc('PlayerCollegeName') + 1, 'PlayerCollegeNameCode', df['PlayerCollegeName'].factorize()[0])
df[['PlayerCollegeNameCode','PlayerCollegeName']].drop_duplicates().sort_index().to_csv('../data/interim/data_PlayerCollegeName.csv', index=False)

In [84]:
df[['NflId','DisplayName']].drop_duplicates().sort_index().to_csv('../data/interim/data_DisplayName.csv', index=False)

In [85]:
dfTeam = pd.DataFrame.from_dict(dctTeamab, orient="index").to_csv("../data/interim/data_TeamAbbreviations.csv")

In [86]:
df.select_dtypes(include=['object']).columns

Index(['Team', 'DisplayName', 'PossessionTeam', 'FieldPosition',
       'OffenseFormation', 'OffensePersonnel', 'DefensePersonnel',
       'PlayDirection', 'PlayerHeight', 'PlayerCollegeName', 'Position',
       'HomeTeamAbbr', 'VisitorTeamAbbr', 'Stadium', 'Location', 'StadiumType',
       'Turf', 'GameWeather', 'WindSpeed', 'WindDirection'],
      dtype='object')

In [87]:
# Location is redundant, remove
df['Location'].unique()
#df.drop(['Location'], axis=1, inplace=True)

array(['Foxborough, MA', 'Orchard Park NY', 'Chicago. IL',
       'Cincinnati, Ohio', 'Cleveland, Ohio', 'Detroit, MI',
       'Houston, Texas', 'Nashville, TN', 'Landover, MD',
       'Los Angeles, Calif.', 'Green Bay, WI', 'Santa Clara, CA',
       'Arlington, Texas', 'Minneapolis, MN', 'Denver, CO',
       'Baltimore, Md.', 'Charlotte, North Carolina',
       'Indianapolis, Ind.', 'Jacksonville, FL', 'Kansas City, MO',
       'New Orleans, LA', 'Pittsburgh', 'Tampa, FL', 'Carson, CA',
       'Oakland, CA', 'Seattle, WA', 'Atlanta, GA', 'East Rutherford, NJ',
       'London, England', 'Chicago, IL', 'Detroit', 'Philadelphia, Pa.',
       'Glendale, AZ', 'Cleveland, OH', 'Foxborough, Ma',
       'E. Rutherford, NJ', 'Miami Gardens, Fla.', 'Houston, TX',
       'London', 'New Orleans, La.', 'Mexico City', 'Baltimore, Maryland',
       'Arlington, TX', 'Jacksonville, Fl', 'Jacksonville, Florida',
       'Pittsburgh, PA', 'Charlotte, NC', 'Cleveland,Ohio',
       'East Rutherford, N.J.',

# Turf Text to Categorical
The turf field can be cleaned up. The field mentions when the field is grass, turf, or a hybrid of both. The attribute has a list of products, so I normalized the data so that it falls under the three categories, and then made a categorical integer field. 

In [88]:
list(df['Turf'].unique())

['Field Turf',
 'A-Turf Titan',
 'Grass',
 'UBU Sports Speed S5-M',
 'Artificial',
 'DD GrassMaster',
 'Natural Grass',
 'UBU Speed Series-S5-M',
 'FieldTurf',
 'FieldTurf 360',
 'Natural grass',
 'grass',
 'Natural',
 'Artifical',
 'FieldTurf360',
 'Naturall Grass',
 'Field turf',
 'SISGrass',
 'Twenty-Four/Seven Turf',
 'natural grass',
 'UBU-Speed Series-S5-M',
 'Twenty Four/Seven Turf',
 'Turf']

In [89]:
# df['Turf']=df['Turf'].replace('Domed','Domed, closed')
df['Turf']=df['Turf'].str.replace(r'^.*\b(?:[Tt]urf)\b.*$','Turf')
df['Turf']=df['Turf'].str.replace(r'^.*\b(?:[Gg]rass)\b.*$','Grass')

df['Turf']=df['Turf'].str.replace(r'^.*SISGrass.*$','Hybrid')
df['Turf']=df['Turf'].str.replace(r'^.*DD GrassMaster.*$','Hybrid')

df['Turf']=df['Turf'].str.replace(r'^.*Turf.*$','Turf')
df['Turf']=df['Turf'].str.replace(r'^.*UBU.*$','Turf')
df['Turf']=df['Turf'].str.replace(r'^.*Artific[i]?al.*$','Turf')
df['Turf']=df['Turf'].str.replace(r'^.*FieldTurf.*$','Turf')

df['Turf']=df['Turf'].str.replace(r'^.*Natural.*$','Grass')



In [90]:
df['Turf'].value_counts(dropna = False)

Grass     377278
Turf      283140
Hybrid     21736
Name: Turf, dtype: int64

In [91]:
TurfCodes = {'Grass':1, 'Turf':2, 'Hybrid':3}
df.insert(df.columns.get_loc('Turf') + 1, 'TurfCode', df['Turf'].map(TurfCodes ))

## Position field corrections
There are 22 players on the field for every play. So every PlayId has 22 entries for each. All that will be focussed on is the running back making the play, but to remove all the other players info seems like a waste. So the position of the player will be collected. So, for example, if in one play, there are 2 defensive tackles on the field at the time of the play, the newly created column of Defensive Tackles will have a 2 in it. 

In [92]:
# Add a new Position Text field for cleaning
df.insert(df.columns.get_loc('Position') + 1, 'PositionText', df['Position'] )

In [93]:
df['PositionText'].value_counts(dropna = False)

CB     74474
WR     71494
T      56266
G      54613
DT     47220
DE     46827
TE     44232
OLB    43333
C      32997
FS     32691
RB     31370
QB     31153
SS     26790
ILB    23029
LB     13989
MLB    13276
NT     10753
OT      8370
OG      4945
FB      4799
DB      4756
S       2656
DL      1134
HB       968
SAF       19
Name: PositionText, dtype: int64

In [94]:
# Strong Safety and Free Safety, and "SAF" are just safety
df['PositionText']=df['PositionText'].replace('SS','S')
df['PositionText']=df['PositionText'].replace('FS','S')
df['PositionText']=df['PositionText'].replace('SAF','S')
# Outside Line Backers, Inside Line backers and Middle Line Backgers to Line Backers
df['PositionText']=df['PositionText'].replace('OLB','LB')
df['PositionText']=df['PositionText'].replace('MLB','LB')
df['PositionText']=df['PositionText'].replace('ILB','LB')
# For Simplicity sake, Defensive backs can be Cornerbacks or Safeties, so I'll set them as Cornerbacks
df['PositionText']=df['PositionText'].replace('DB','CB')
# Defensive linemen are gouped as linemen
df['PositionText']=df['PositionText'].replace('DL','LB')
# Offensive tacles and offensive guards are grouped together for simplicity
df['PositionText']=df['PositionText'].replace('T','OT')
df['PositionText']=df['PositionText'].replace('G','OG')

In [95]:
df['PositionText']=df['PositionText'].replace('QB','Quarterbacks')
df['PositionText']=df['PositionText'].replace('RB','RunningBacks')
df['PositionText']=df['PositionText'].replace('FB','Fullbacks')
df['PositionText']=df['PositionText'].replace('WR','WideReceivers')
df['PositionText']=df['PositionText'].replace('TE','TightEnds')
df['PositionText']=df['PositionText'].replace('HB','Halfbacks')

df['PositionText']=df['PositionText'].replace('CB','Cornerbacks')
df['PositionText']=df['PositionText'].replace('S','Safeties')
df['PositionText']=df['PositionText'].replace('DT','DefensiveTackles')
df['PositionText']=df['PositionText'].replace('NT','NoseTackles')
df['PositionText']=df['PositionText'].replace('DE','DefensiveEnds')
df['PositionText']=df['PositionText'].replace('LB','LineBackers')

df['PositionText']=df['PositionText'].replace('C','Centers')
df['PositionText']=df['PositionText'].replace('OT','OffensiveTackles')
df['PositionText']=df['PositionText'].replace('OG','OffensiveGuards')


In [96]:
# Normalizing the newly created PositionText field to add 15 new attributes, containing the counts of each position
# This will replace the OffensivePersonnel and DefensePersonnel attributes.
dfpositions = df.groupby('PlayId')['PositionText'].value_counts().unstack().fillna(0).astype('int64')

In [97]:
# Separating the dataset so that the player making the play is the focus
dfrun = df[df["NflIdRusher"] == df["NflId"]]
dfrun.shape

(31007, 69)

In [98]:
# Merging the normalized data with the newly extracted data
dfmerge = pd.merge(dfrun, dfpositions, on='PlayId')

### Save the new files to the appropriate directories
One final check of the data, dropping some of the obviously unneeded columns, and saving the first stage of data into a csv file. It looks like all of the fields are full with no N/A fields. Also, the data went from 45 columns to 83 columns, so a lot of attributes are going to be dropped in the next stages. 

In [99]:
dfmerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31007 entries, 0 to 31006
Data columns (total 84 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   GameId                  31007 non-null  int64              
 1   PlayId                  31007 non-null  int64              
 2   Team                    31007 non-null  object             
 3   isHomeTeam              31007 non-null  int32              
 4   X                       31007 non-null  float64            
 5   Y                       31007 non-null  float64            
 6   S                       31007 non-null  float64            
 7   A                       31007 non-null  float64            
 8   Dis                     31007 non-null  float64            
 9   Orientation             31007 non-null  float64            
 10  Dir                     31007 non-null  float64            
 11  NflId                   31007 non-null  i

In [100]:
# Dropping the columns that are definitely not needed moving forward
dfmerge.drop(['Stadium','Humidity','Location','OffensePersonnel', 'DefensePersonnel','GameWeather'], axis=1, inplace=True)
dfmerge.drop(['PlayerHeight','PlayerHeightFt','PlayerHeightIn'], axis=1, inplace=True)
dfmerge.drop(['YardLine','LeadDeficit'], axis=1, inplace=True)


In [101]:
dfmerge.to_csv("../data/processed/data_cleaned.csv", index=False)